In [112]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

#Define input image dimensions
#Large images take too much time and resources.
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

In [113]:
#Given input of noise (latent) vector, the Generator produces an image.
def build_generator():

    noise_shape = (100,) #1D array of size 100 (latent vector / noise)

#Define your generator network 
#Here we are only using Dense layers. But network can be complicated based
#on the application. For example, you can use VGG for super res. GAN.         

    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Dense(256, input_shape=noise_shape))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.BatchNormalization(momentum=0.8))
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.BatchNormalization(momentum=0.8))
    model.add(tf.keras.layers.Dense(1024))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.BatchNormalization(momentum=0.8))
    
    model.add(tf.keras.layers.Dense(np.prod(img_shape), activation='tanh'))
    model.add(tf.keras.layers.Reshape(img_shape))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)    #Generated image

    return Model(noise, img)


In [114]:
#Given an input image, the Discriminator outputs the likelihood of the image being real.
#Binary classification - true or false (we're calling it validity)

def build_discriminator():


    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Flatten(input_shape=img_shape))
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

In [115]:
#The validity is the Discriminator’s guess of input being real or not.


#Now that we have constructed our two models it’s time to pit them against each other.
#We do this by defining a training function, loading the data set, re-scaling our training
#images and setting the ground truths. 
def train(epochs, batch_size=128, save_interval=50):

    # Load the dataset
    (X_train, X_test), (y_train, y_test) = mnist.load_data()

    # Convert to float and Rescale -1 to 1 (Can also do 0 to 1)
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5

#Add channels dimension. As the input to our gen and discr. has a shape 28x28x1.
    X_train = np.expand_dims(X_train, axis=3) 

    half_batch = int(batch_size / 2)
    
#We then loop through a number of epochs to train our Discriminator by first selecting
#a random batch of images from our true dataset, generating a set of images from our
#Generator, feeding both set of images into our Discriminator, and finally setting the
#loss parameters for both the real and fake images, as well as the combined loss. 
    
    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch of real images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]

 
        noise = np.random.normal(0, 1, (half_batch, 100))

        # Generate a half batch of fake images
        gen_imgs = generator.predict(noise)

        # Train the discriminator on real and fake images, separately
        #Research showed that separate training is more effective. 
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    #take average loss from real and fake images. 
    #
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 

#And within the same loop we train our Generator, by setting the input noise and
#ultimately training the Generator to have the Discriminator label its samples as valid
#by specifying the gradient loss.
        # ---------------------
        #  Train Generator
        # ---------------------
#Create noise vectors as input for generator. 
#Create as many noise vectors as defined by the batch size. 
#Based on normal distribution. Output will be of size (batch size, 100)
        noise = np.random.normal(0, 1, (batch_size, 100)) 

        # The generator wants the discriminator to label the generated samples
        # as valid (ones)
        #This is where the genrator is trying to trick discriminator into believing
        #the generated image is true (hence value of 1 for y)
        valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size

        # Generator is part of combined where it got directly linked with the discriminator
        # Train the generator with noise as x and 1 as y. 
        # Again, 1 as the output as it is adversarial and if generator did a great
        #job of folling the discriminator then the output would be 1 (true)
        g_loss = combined.train_on_batch(noise, valid_y)


#Additionally, in order for us to keep track of our training process, we print the
#progress and save the sample image output depending on the epoch interval specified.  
# Plot the progress
        
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            save_imgs(epoch)

In [116]:
#when the specific sample_interval is hit, we call the
#sample_image function. Which looks as follows.

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()
#This function saves our images for us to view

In [117]:
#Let us also define our optimizer for easy use later on.
#That way if you change your mind, you can change it easily here
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.

#Build and compile the discriminator first. 
#Generator will be trained as part of the combined model, later. 
#pick the loss function and the type of metric to keep track.                 
#Binary cross entropy as we are doing prediction and it is a better
#loss function compared to MSE or other. 
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

#build and compile our Discriminator, pick the loss function

#SInce we are only generating (faking) images, let us not track any metrics.
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

##This builds the Generator and defines the input noise. 
#In a GAN the Generator network takes noise z as an input to produce its images.  
z = Input(shape=(100,))   #Our random input to the generator
img = generator(z)

#This ensures that when we combine our networks we only train the Generator.
#While generator training we do not want discriminator weights to be adjusted. 
#This Doesn't affect the above descriminator training.     
discriminator.trainable = False  

#This specifies that our Discriminator will take the images generated by our Generator
#and true dataset and set its output to a parameter called valid, which will indicate
#whether the input is real or not.  
valid = discriminator(img)  #Validity check on the generated image


#Here we combined the models and also set our loss function and optimizer. 
#Again, we are only training the generator here. 
#The ultimate goal here is for the Generator to fool the Discriminator.  
# The combined model  (stacked generator and discriminator) takes
# noise as input => generates images => determines validity

combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)


train(epochs=10000, batch_size=32, save_interval=1000)

#Save model for future use to generate fake images
#Not tested yet... make sure right model is being saved..
#Compare with GAN4

generator.save('generator_model.h5')  #Test the model on GAN4_predict...
#Change epochs back to 30K
                
#Epochs dictate the number of backward and forward propagations, the batch_size
#indicates the number of training samples per backward/forward propagation, and the
#sample_interval specifies after how many epochs we call our sample_image function.

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
________________________________________________

94 [D loss: 0.012921, acc.: 100.00%] [G loss: 4.544488]
95 [D loss: 0.009975, acc.: 100.00%] [G loss: 4.560637]
96 [D loss: 0.011581, acc.: 100.00%] [G loss: 4.596752]
97 [D loss: 0.008961, acc.: 100.00%] [G loss: 4.653012]
98 [D loss: 0.011042, acc.: 100.00%] [G loss: 4.471616]
99 [D loss: 0.012980, acc.: 100.00%] [G loss: 4.643322]
100 [D loss: 0.011895, acc.: 100.00%] [G loss: 4.626582]
101 [D loss: 0.006523, acc.: 100.00%] [G loss: 4.602665]
102 [D loss: 0.007169, acc.: 100.00%] [G loss: 4.651532]
103 [D loss: 0.022872, acc.: 100.00%] [G loss: 4.467365]
104 [D loss: 0.011220, acc.: 100.00%] [G loss: 4.683184]
105 [D loss: 0.012028, acc.: 100.00%] [G loss: 4.282192]
106 [D loss: 0.014551, acc.: 100.00%] [G loss: 4.664246]
107 [D loss: 0.009117, acc.: 100.00%] [G loss: 4.524731]
108 [D loss: 0.018984, acc.: 100.00%] [G loss: 4.774197]
109 [D loss: 0.018326, acc.: 100.00%] [G loss: 4.460105]
110 [D loss: 0.027578, acc.: 100.00%] [G loss: 4.798082]
111 [D loss: 0.011559, acc.: 100.00%]

239 [D loss: 0.482804, acc.: 84.38%] [G loss: 1.295389]
240 [D loss: 0.703203, acc.: 50.00%] [G loss: 1.163684]
241 [D loss: 0.649478, acc.: 62.50%] [G loss: 1.375166]
242 [D loss: 0.711927, acc.: 56.25%] [G loss: 1.113358]
243 [D loss: 0.749228, acc.: 50.00%] [G loss: 0.795871]
244 [D loss: 0.449073, acc.: 84.38%] [G loss: 1.410870]
245 [D loss: 0.852095, acc.: 40.62%] [G loss: 0.930890]
246 [D loss: 0.755320, acc.: 43.75%] [G loss: 0.855020]
247 [D loss: 0.599442, acc.: 65.62%] [G loss: 1.066325]
248 [D loss: 0.795130, acc.: 46.88%] [G loss: 1.038219]
249 [D loss: 0.525204, acc.: 65.62%] [G loss: 1.405562]
250 [D loss: 1.084175, acc.: 25.00%] [G loss: 0.557779]
251 [D loss: 0.590490, acc.: 56.25%] [G loss: 1.120157]
252 [D loss: 0.621477, acc.: 68.75%] [G loss: 1.172658]
253 [D loss: 0.898607, acc.: 34.38%] [G loss: 0.629642]
254 [D loss: 0.496802, acc.: 62.50%] [G loss: 1.130041]
255 [D loss: 0.937159, acc.: 31.25%] [G loss: 0.804385]
256 [D loss: 0.598535, acc.: 59.38%] [G loss: 0.

386 [D loss: 0.672879, acc.: 50.00%] [G loss: 0.634346]
387 [D loss: 0.668381, acc.: 50.00%] [G loss: 0.642003]
388 [D loss: 0.647678, acc.: 53.12%] [G loss: 0.657578]
389 [D loss: 0.671118, acc.: 46.88%] [G loss: 0.683193]
390 [D loss: 0.664998, acc.: 53.12%] [G loss: 0.675506]
391 [D loss: 0.682133, acc.: 53.12%] [G loss: 0.665803]
392 [D loss: 0.681237, acc.: 43.75%] [G loss: 0.661094]
393 [D loss: 0.671430, acc.: 46.88%] [G loss: 0.665699]
394 [D loss: 0.644459, acc.: 50.00%] [G loss: 0.661936]
395 [D loss: 0.673224, acc.: 46.88%] [G loss: 0.666262]
396 [D loss: 0.673975, acc.: 46.88%] [G loss: 0.662829]
397 [D loss: 0.627966, acc.: 56.25%] [G loss: 0.688360]
398 [D loss: 0.648183, acc.: 50.00%] [G loss: 0.707865]
399 [D loss: 0.655051, acc.: 46.88%] [G loss: 0.708753]
400 [D loss: 0.669579, acc.: 50.00%] [G loss: 0.698200]
401 [D loss: 0.639504, acc.: 53.12%] [G loss: 0.688590]
402 [D loss: 0.688630, acc.: 40.62%] [G loss: 0.671781]
403 [D loss: 0.703276, acc.: 43.75%] [G loss: 0.

533 [D loss: 0.663666, acc.: 53.12%] [G loss: 0.720458]
534 [D loss: 0.673983, acc.: 50.00%] [G loss: 0.724410]
535 [D loss: 0.640106, acc.: 59.38%] [G loss: 0.710739]
536 [D loss: 0.650899, acc.: 53.12%] [G loss: 0.715227]
537 [D loss: 0.659559, acc.: 56.25%] [G loss: 0.722040]
538 [D loss: 0.668731, acc.: 56.25%] [G loss: 0.713468]
539 [D loss: 0.647928, acc.: 50.00%] [G loss: 0.729670]
540 [D loss: 0.660453, acc.: 56.25%] [G loss: 0.716842]
541 [D loss: 0.691951, acc.: 53.12%] [G loss: 0.720589]
542 [D loss: 0.681240, acc.: 46.88%] [G loss: 0.714090]
543 [D loss: 0.626875, acc.: 59.38%] [G loss: 0.694514]
544 [D loss: 0.660287, acc.: 50.00%] [G loss: 0.668868]
545 [D loss: 0.652618, acc.: 56.25%] [G loss: 0.686133]
546 [D loss: 0.676015, acc.: 56.25%] [G loss: 0.668446]
547 [D loss: 0.629584, acc.: 62.50%] [G loss: 0.677520]
548 [D loss: 0.640665, acc.: 56.25%] [G loss: 0.682461]
549 [D loss: 0.648103, acc.: 53.12%] [G loss: 0.669737]
550 [D loss: 0.649722, acc.: 65.62%] [G loss: 0.

...

9332 [D loss: 0.700592, acc.: 56.25%] [G loss: 0.857588]
9333 [D loss: 0.575503, acc.: 81.25%] [G loss: 0.827666]
9334 [D loss: 0.603463, acc.: 71.88%] [G loss: 0.777025]
9335 [D loss: 0.778303, acc.: 37.50%] [G loss: 0.853803]
9336 [D loss: 0.739503, acc.: 50.00%] [G loss: 0.875249]
9337 [D loss: 0.686206, acc.: 62.50%] [G loss: 0.869782]
9338 [D loss: 0.725508, acc.: 53.12%] [G loss: 0.849668]
9339 [D loss: 0.646449, acc.: 68.75%] [G loss: 0.836326]
9340 [D loss: 0.724766, acc.: 53.12%] [G loss: 0.883696]
9341 [D loss: 0.695913, acc.: 50.00%] [G loss: 0.816603]
9342 [D loss: 0.741171, acc.: 50.00%] [G loss: 0.863277]
9343 [D loss: 0.583907, acc.: 71.88%] [G loss: 0.891265]
9344 [D loss: 0.632027, acc.: 71.88%] [G loss: 0.873112]
9345 [D loss: 0.634855, acc.: 62.50%] [G loss: 0.915416]
9346 [D loss: 0.696448, acc.: 40.62%] [G loss: 0.884250]
9347 [D loss: 0.656555, acc.: 59.38%] [G loss: 0.869389]
9348 [D loss: 0.717700, acc.: 56.25%] [G loss: 0.873007]
9349 [D loss: 0.657788, acc.: 6

9476 [D loss: 0.687558, acc.: 56.25%] [G loss: 0.897837]
9477 [D loss: 0.613644, acc.: 56.25%] [G loss: 0.907252]
9478 [D loss: 0.662304, acc.: 59.38%] [G loss: 0.862702]
9479 [D loss: 0.714002, acc.: 62.50%] [G loss: 0.825931]
9480 [D loss: 0.663337, acc.: 59.38%] [G loss: 0.830071]
9481 [D loss: 0.703155, acc.: 62.50%] [G loss: 0.857445]
9482 [D loss: 0.709351, acc.: 59.38%] [G loss: 0.866365]
9483 [D loss: 0.654710, acc.: 56.25%] [G loss: 0.851981]
9484 [D loss: 0.626643, acc.: 56.25%] [G loss: 0.894743]
9485 [D loss: 0.675001, acc.: 59.38%] [G loss: 0.896138]
9486 [D loss: 0.664781, acc.: 50.00%] [G loss: 0.883278]
9487 [D loss: 0.672195, acc.: 68.75%] [G loss: 0.875654]
9488 [D loss: 0.719651, acc.: 46.88%] [G loss: 0.863483]
9489 [D loss: 0.677405, acc.: 56.25%] [G loss: 0.803314]
9490 [D loss: 0.711971, acc.: 56.25%] [G loss: 0.843850]
9491 [D loss: 0.617698, acc.: 62.50%] [G loss: 0.857685]
9492 [D loss: 0.699733, acc.: 50.00%] [G loss: 0.932564]
9493 [D loss: 0.677957, acc.: 5

9620 [D loss: 0.693150, acc.: 50.00%] [G loss: 0.850996]
9621 [D loss: 0.694988, acc.: 59.38%] [G loss: 0.852499]
9622 [D loss: 0.726674, acc.: 56.25%] [G loss: 0.830409]
9623 [D loss: 0.664901, acc.: 59.38%] [G loss: 0.816053]
9624 [D loss: 0.643079, acc.: 65.62%] [G loss: 0.909299]
9625 [D loss: 0.671879, acc.: 59.38%] [G loss: 0.898689]
9626 [D loss: 0.741369, acc.: 56.25%] [G loss: 0.881654]
9627 [D loss: 0.600160, acc.: 71.88%] [G loss: 0.939676]
9628 [D loss: 0.724357, acc.: 56.25%] [G loss: 0.831109]
9629 [D loss: 0.709228, acc.: 56.25%] [G loss: 0.808800]
9630 [D loss: 0.728067, acc.: 50.00%] [G loss: 0.801755]
9631 [D loss: 0.625507, acc.: 59.38%] [G loss: 0.889048]
9632 [D loss: 0.602337, acc.: 65.62%] [G loss: 0.864560]
9633 [D loss: 0.732943, acc.: 50.00%] [G loss: 0.805069]
9634 [D loss: 0.771514, acc.: 53.12%] [G loss: 0.770788]
9635 [D loss: 0.667898, acc.: 50.00%] [G loss: 0.889944]
9636 [D loss: 0.716763, acc.: 40.62%] [G loss: 0.833613]
9637 [D loss: 0.650634, acc.: 6

9764 [D loss: 0.658644, acc.: 59.38%] [G loss: 0.866530]
9765 [D loss: 0.655485, acc.: 62.50%] [G loss: 0.965536]
9766 [D loss: 0.632088, acc.: 65.62%] [G loss: 0.874992]
9767 [D loss: 0.623176, acc.: 68.75%] [G loss: 0.837195]
9768 [D loss: 0.719584, acc.: 46.88%] [G loss: 0.850848]
9769 [D loss: 0.717882, acc.: 59.38%] [G loss: 0.769125]
9770 [D loss: 0.702296, acc.: 43.75%] [G loss: 0.789746]
9771 [D loss: 0.643587, acc.: 68.75%] [G loss: 0.845202]
9772 [D loss: 0.677627, acc.: 53.12%] [G loss: 0.780045]
9773 [D loss: 0.675747, acc.: 65.62%] [G loss: 0.893285]
9774 [D loss: 0.563940, acc.: 78.12%] [G loss: 0.862678]
9775 [D loss: 0.699609, acc.: 65.62%] [G loss: 0.841316]
9776 [D loss: 0.576110, acc.: 78.12%] [G loss: 0.803077]
9777 [D loss: 0.587777, acc.: 68.75%] [G loss: 0.890042]
9778 [D loss: 0.840766, acc.: 43.75%] [G loss: 0.824016]
9779 [D loss: 0.651235, acc.: 59.38%] [G loss: 0.877836]
9780 [D loss: 0.686708, acc.: 50.00%] [G loss: 0.885889]
9781 [D loss: 0.704352, acc.: 5

9908 [D loss: 0.664460, acc.: 59.38%] [G loss: 0.815882]
9909 [D loss: 0.651868, acc.: 65.62%] [G loss: 0.900617]
9910 [D loss: 0.682010, acc.: 59.38%] [G loss: 0.884880]
9911 [D loss: 0.741853, acc.: 46.88%] [G loss: 0.860141]
9912 [D loss: 0.612699, acc.: 68.75%] [G loss: 0.806004]
9913 [D loss: 0.602132, acc.: 59.38%] [G loss: 0.829790]
9914 [D loss: 0.720192, acc.: 53.12%] [G loss: 0.804832]
9915 [D loss: 0.666191, acc.: 59.38%] [G loss: 0.839399]
9916 [D loss: 0.630943, acc.: 68.75%] [G loss: 0.937595]
9917 [D loss: 0.676911, acc.: 50.00%] [G loss: 0.928802]
9918 [D loss: 0.604692, acc.: 65.62%] [G loss: 0.917976]
9919 [D loss: 0.711662, acc.: 46.88%] [G loss: 0.928857]
9920 [D loss: 0.642580, acc.: 65.62%] [G loss: 0.889385]
9921 [D loss: 0.799956, acc.: 34.38%] [G loss: 0.799472]
9922 [D loss: 0.643797, acc.: 56.25%] [G loss: 0.883005]
9923 [D loss: 0.756787, acc.: 46.88%] [G loss: 0.902294]
9924 [D loss: 0.678786, acc.: 50.00%] [G loss: 0.932796]
9925 [D loss: 0.642036, acc.: 6